1. Which product categories generate the highest revenue?
-Use $group to calculate revenue by category.
-Sort the results in descending order.


In [ ]:
db.getCollection('order_items').aggregate(
  [
    {
      $lookup: {
        from: 'products',
        localField: 'product_id',
        foreignField: '_id',
        as: 'product_details'
      }
    },
    {
      $unwind: {
        path: '$product_details',
        includeArrayIndex: 'includeArrayIndex',
        preserveNullAndEmptyArrays: true
      }
    },
    {
      $sort: {
        order_id: 1,
        'product_details.price': -1
      }
    },
    {
      $group: {
        _id: '$order_id',
        top_products: {
          $push: {
            product_id: '$product_id',
            products_name:
              '$product_details.products_name',
            price: '$product_details.price'
          }
        }
      }
    },
    {
      $project: {
        top_products: {
          $slice: ['$top_products', 3]
        }
      }
    }
  ],
  { maxTimeMS: 60000, allowDiskUse: true }
);

2.What is the average delivery time for orders?
-Calculate the difference between order_date and delivery_date.
-Use $group to calculate the average delivery time.


In [ ]:
db.getCollection('orders').aggregate(
  [
    {
      $project: {
        order_id: 1,
        customer_id: 1,
        order_date: 1,
        delivery_date: 1,
        delivery_time: {
          $subtract: [
            '$delivery_date',
            '$order_date'
          ]
        }
      }
    },
    {
      $group: {
        _id: 'delievery_time',
        average_delivery_time: {
          $avg: '$delivery_time'
        }
      }
    },
    {
      $project: {
        average_delivery_time_in_days: {
          $divide: [
            '$average_delivery_time',
            86400000
          ]
        },
        average_delivery_time_in_hours: {
          $mod: [
            {
              $divide: [
                '$average_delivery_time',
                3600000
              ]
            },
            24
          ]
        },
        average_delivery_time_in_minutes: {
          $mod: [
            {
              $divide: [
                '$average_delivery_time',
                60000
              ]
            },
            60
          ]
        },
        average_delivery_time_in_seconds: {
          $mod: [
            {
              $divide: [
                '$average_delivery_time',
                1000
              ]
            },
            60
          ]
        }
      }
    }
  ],
  { maxTimeMS: 60000, allowDiskUse: true }
);

3.Which states have the highest number of customers?
-Use $group to count customers by state.
-Sort the results in descending order.

In [ ]:
db.getCollection('customers').aggregate(
  [
    {
      $project: {
        address_parts: {
          $split: ['$address', ', ']
        }
      }
    },
    {
      $project: {
        city: {
          $arrayElemAt: ['$address_parts', 0]
        },
        state: {
          $arrayElemAt: ['$address_parts', 1]
        },
        zip_code: {
          $arrayElemAt: ['$address_parts', 2]
        }
      }
    },
    {
      $group: {
        _id: '$city',
        customer_count: { $sum: 1 }
      }
    },
    { $sort: { customer_count: -1 } }
  ],
  { maxTimeMS: 60000, allowDiskUse: true }
);

4.What are the top 3 most expensive products sold in each order?
-Use $lookup and $sort to find the top products in each order.


In [ ]:
db.getCollection('order_items').aggregate(
  [
    {
      $lookup: {
        from: 'products',
        localField: 'product_id',
        foreignField: '_id',
        as: 'product_details'
      }
    },
    {
      $unwind: {
        path: '$product_details',
        includeArrayIndex: 'includeArrayIndex',
        preserveNullAndEmptyArrays: true
      }
    },
    {
      $sort: {
        order_id: 1,
        'product_details.price': -1
      }
    },
    {
      $group: {
        _id: '$order_id',
        top_products: {
          $push: {
            product_id: '$product_id',
            products_name:
              '$product_details.products_name',
            price: '$product_details.price'
          }
        }
      }
    },
    {
      $project: {
        top_products: {
          $slice: ['$top_products', 3]
        }
      }
    }
  ],
  { maxTimeMS: 60000, allowDiskUse: true }
);